## Sprint11 DLフレームワーク入門
・フレームワークのコードを読めるようにする     
・フレームワークを習得し続けられるようになる     
・理論を知っている範囲をフレームワークで動かす

【目的としないこと】    
・各フレームワークの細かいノウハウの習得   
・デプロイなど動かしてみたその先    
・新たに見かけた理論の詳細な理解

## Irisの2値分類のサンプルコード(Tensorflowのディープラーニング）

In [113]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="/Users/andoutakaaki/DIC_study/sprint3/Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits：与えられたシグモイドクロスエントロピーを計算するlogits。
# 個々のクラスが独立しており、互いに排他的ではない離散分類タスクの確率エラーを測定
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
# tf.sign：数値の符号の要素的な指示を返す
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(Y - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 21.1886, val_loss : 10.0427, acc : 1.000, val_acc : 1.000
Epoch 1, loss : 0.0006, val_loss : 4.2959, acc : 1.000, val_acc : 1.000
Epoch 2, loss : 0.0103, val_loss : 2.1422, acc : 1.000, val_acc : 1.000
Epoch 3, loss : 0.0298, val_loss : 5.5458, acc : 1.000, val_acc : 1.000
Epoch 4, loss : 0.5795, val_loss : 5.4871, acc : 1.000, val_acc : 1.000
Epoch 5, loss : 0.7384, val_loss : 6.6582, acc : 1.000, val_acc : 1.000
Epoch 6, loss : 1.3283, val_loss : 7.4705, acc : 1.000, val_acc : 1.000
Epoch 7, loss : 0.0030, val_loss : 5.0470, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.8645, val_loss : 6.9551, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.0000, val_loss : 3.3107, acc : 1.000, val_acc : 1.000
test_acc : 1.000


loss :訓練データの損失    
val_loss　：バリデーションデータセットの損失     
acc :訓練データの精度     
val_acc :バリデーションデータセットの精度   
test_acc :テストデータの精度

## mnist(kerasのディープラーニング)

In [2]:
# kerasでmnistのDL
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from sklearn.datasets import fetch_mldata
from __future__ import print_function
import keras
mnist_dir = "/Users/andoutakaaki/DIC_study/sprint9/mnist_data/"
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 230
num_classes = 10
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
mnist = fetch_mldata('MNIST original', data_home=mnist_dir)
# the data, split between train and test sets
x_train, x_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.2, random_state=0)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (56000, 28, 28, 1)
56000 train samples
14000 test samples
Train on 56000 samples, validate on 14000 samples
Epoch 1/5
56000/56000 [==============================] - 144s 3ms/step - loss: 0.3558 - acc: 0.8911 - val_loss: 0.0898 - val_acc: 0.9726

## mnist(Tensorflowのディープラーニング）

In [45]:
# TensorfiowでmnistのDL
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.datasets import fetch_mldata

mnist_dir = "/Users/andoutakaaki/DIC_study/sprint9/mnist_data/"

# MNISTの読み込み
mnist = fetch_mldata('MNIST original', data_home=mnist_dir)

# trainとtestに分割
# MNISTの読み込み
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.2)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ラベルをint型にしておく
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)
y_val = y_val.astype(np.int)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_val = X_val.astype(np.float)
X_train /= 255
X_test /= 255
X_val /= 255

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
y_val_one_hot = enc.transform(y_val[:, np.newaxis])

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc)) 
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.5588, val_loss : 1.4502, acc : 0.980, val_acc : 0.953
Epoch 1, loss : 0.2847, val_loss : 0.8052, acc : 0.990, val_acc : 0.970
Epoch 2, loss : 0.2528, val_loss : 0.4505, acc : 0.990, val_acc : 0.979
Epoch 3, loss : 0.0155, val_loss : 0.3623, acc : 1.000, val_acc : 0.983
Epoch 4, loss : 0.0313, val_loss : 0.3717, acc : 1.000, val_acc : 0.983
Epoch 5, loss : 0.0333, val_loss : 0.3551, acc : 1.000, val_acc : 0.984
Epoch 6, loss : 0.0111, val_loss : 0.3290, acc : 1.000, val_acc : 0.986
Epoch 7, loss : 0.0046, val_loss : 0.3032, acc : 1.000, val_acc : 0.987
Epoch 8, loss : 0.0773, val_loss : 0.3326, acc : 0.990, val_acc : 0.986
Epoch 9, loss : 0.0145, val_loss : 0.3204, acc : 1.000, val_acc : 0.987
test_acc : 0.987


## Irisの３値分類(Tensorflowのディープラーニング）

In [82]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類するコード
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
dataset_path ="/Users/andoutakaaki/DIC_study/sprint3/Iris.csv"
df = pd.read_csv(dataset_path)

# データフレームから条件抽出

y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y) 
y = np.identity(3)[y]
print(y.shape)
X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._counter = 0
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __iter__(self):
        return self
    def __next__(self):
        if self._counter >= self._stop:
            self._counter = 0
            raise StopIteration()

        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits：与えられたシグモイドクロスエントロピーを計算するlogits。
# 個々のクラスが独立しており、互いに排他的ではない離散分類タスクの確率エラーを測定
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
# tf.sign：数値の符号の要素的な指示を返す
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        print(mini_batch_x,mini_batch_y.shape)
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

(150, 3)
[[4.9 2.4 3.3 1. ]
 [6.3 2.7 4.9 1.8]
 [6.8 2.8 4.8 1.4]
 [5.7 4.4 1.5 0.4]
 [6.3 2.3 4.4 1.3]
 [6.3 2.5 4.9 1.5]] (6, 3)
Epoch 0, loss : 40.1334, val_loss : 19.0472, acc : 0.333, val_acc : 0.694
[[6.9 3.1 4.9 1.5]
 [6.2 2.9 4.3 1.3]
 [5.8 2.6 4.  1.2]
 [6.3 3.3 4.7 1.6]
 [6.3 3.3 6.  2.5]
 [6.4 3.2 5.3 2.3]] (6, 3)
Epoch 1, loss : 0.8430, val_loss : 1.9871, acc : 0.778, val_acc : 0.889
[[6.9 3.1 4.9 1.5]
 [6.2 2.9 4.3 1.3]
 [5.8 2.6 4.  1.2]
 [6.3 3.3 4.7 1.6]
 [6.3 3.3 6.  2.5]
 [6.4 3.2 5.3 2.3]] (6, 3)
Epoch 2, loss : 3.2981, val_loss : 4.7246, acc : 0.778, val_acc : 0.750
[[6.9 3.1 4.9 1.5]
 [6.2 2.9 4.3 1.3]
 [5.8 2.6 4.  1.2]
 [6.3 3.3 4.7 1.6]
 [6.3 3.3 6.  2.5]
 [6.4 3.2 5.3 2.3]] (6, 3)
Epoch 3, loss : 0.0000, val_loss : 0.1330, acc : 1.000, val_acc : 0.972
[[6.9 3.1 4.9 1.5]
 [6.2 2.9 4.3 1.3]
 [5.8 2.6 4.  1.2]
 [6.3 3.3 4.7 1.6]
 [6.3 3.3 6.  2.5]
 [6.4 3.2 5.3 2.3]] (6, 3)
Epoch 4, loss : 0.0000, val_loss : 0.3556, acc : 1.000, val_acc : 0.972
[[6.9 3.1 4.9 1.5]


## House Prices(Tensorflowでディープラーニング)

In [111]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

df = pd.read_csv("/Users/andoutakaaki/DIC_study/sprint8/train.csv")
df =df[['SalePrice','GrLivArea','YearBuilt']]

X = df[['GrLivArea','YearBuilt']].values  # 説明変数は'GrLivArea','YearBuilt'
y = df[['SalePrice']].values  # 目的変数は'SalePrice'
y = np.array(y)
X = np.array(X)
print(X.shape,y.shape)
 
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(1460, 2) (1460, 1)
(934, 2)
(292, 2)
(234, 2)
(934, 1)
(292, 1)
(234, 1)


In [115]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.0001
batch_size = 10
num_epochs = 10

n_hidden1 = 100
n_hidden2 = 50
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean((tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits)))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
#correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.abs(Y - logits))

# variableの初期化
init = tf.global_variables_initializer()


# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        print(mini_batch_x.shape,mini_batch_y.shape)
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc)) 
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))






(4, 4) (4, 1)
Epoch 0, loss : 25.2171, val_loss : 12.9125, acc : 35.085, val_acc : 34.310
(4, 4) (4, 1)
Epoch 1, loss : 19.7507, val_loss : 10.3233, acc : 28.015, val_acc : 28.131
(4, 4) (4, 1)
Epoch 2, loss : 14.2837, val_loss : 7.7351, acc : 20.948, val_acc : 21.961
(4, 4) (4, 1)
Epoch 3, loss : 9.1824, val_loss : 5.2406, acc : 14.137, val_acc : 16.017
(4, 4) (4, 1)
Epoch 4, loss : 5.5717, val_loss : 3.1910, acc : 10.125, val_acc : 11.018
(4, 4) (4, 1)
Epoch 5, loss : 3.0011, val_loss : 2.1224, acc : 7.395, val_acc : 8.002
(4, 4) (4, 1)
Epoch 6, loss : 1.9467, val_loss : 1.8314, acc : 6.758, val_acc : 6.949
(4, 4) (4, 1)
Epoch 7, loss : 1.5826, val_loss : 1.7721, acc : 6.824, val_acc : 6.894
(4, 4) (4, 1)
Epoch 8, loss : 1.5306, val_loss : 1.7428, acc : 6.918, val_acc : 7.050
(4, 4) (4, 1)
Epoch 9, loss : 1.6299, val_loss : 1.7325, acc : 7.022, val_acc : 7.270
test_acc : 9.271
